A ideia é criar um pipeline **realista**, mas didático — com **camadas Bronze → Silver → Gold (Fato e Dimensão)** — usando **Databricks + Spark**.

## 💾 1️⃣ Cenário proposto

Vamos simular uma pipeline de **vendas de e-commerce** — um clássico e ótimo exemplo para aprendizado.

**Fonte (arquivo de entrada):**

* Um arquivo CSV de **pedidos de compra** contendo transações brutas exportadas do sistema de vendas.

### 🧾 Exemplo de arquivo (`orders_raw.csv`)

| order_id | order_date | customer_id | product_id | quantity | unit_price | country   | payment_method | status    |
| -------- | ---------- | ----------- | ---------- | -------- | ---------- | --------- | -------------- | --------- |
| 1001     | 2024-05-10 | C001        | P001       | 2        | 150.00     | Brazil    | Credit Card    | Delivered |
| 1002     | 2024-05-11 | C002        | P002       | 1        | 80.00      | Argentina | PIX            | Cancelled |
| 1003     | 2024-05-11 | C001        | P003       | 3        | 50.00      | Brazil    | Credit Card    | Delivered |
| 1004     | 2024-05-12 | C003        | P001       | 1        | 150.00     | Chile     | Debit Card     | Delivered |

## ⚙️ 2️⃣ Objetivos da Pipeline

| Camada     | Objetivo                 | Descrição                                                                       |
| ---------- | ------------------------ | ------------------------------------------------------------------------------- |
| **Bronze** | Ingestão                 | Ingerir o arquivo *raw*, armazenar sem transformação (data lake).               |
| **Silver** | Limpeza e enriquecimento | Tratar nulos, tipos, normalizar países, calcular `total_price`.                 |
| **Gold**   | Modelagem dimensional    | Criar tabelas **Dimensão** (Cliente, Produto, Tempo, País) e **Fato** (Vendas). |

## 🧱 3️⃣ Estrutura de pastas no Data Lake

```
/mnt/datalake/
    ├── bronze/
    │     └── orders/
    ├── silver/
    │     └── orders_clean/
    └── gold/
          ├── dim_customer/
          ├── dim_product/
          ├── dim_date/
          ├── dim_country/
          └── fact_sales/
```

## 💡 4️⃣ Transformações esperadas

### 🔸 Bronze

* Leitura direta do CSV.
* Armazenar em formato **Delta** (`bronze.orders`).

```python
df_bronze = (
    spark.read.option("header", True)
    .csv("/mnt/raw/orders_raw.csv")
)
df_bronze.write.format("delta").mode("overwrite").save("/mnt/datalake/bronze/orders")
```

### 🔸 Silver

Limpezas e enriquecimentos:

* Converter tipos (`quantity` → int, `unit_price` → float).
* Normalizar `country` e `status`.
* Criar campo `total_price = quantity * unit_price`.
* Filtrar apenas `status = 'Delivered'`.

```python
from pyspark.sql.functions import col, upper, trim, when

df_silver = (
    df_bronze
    .withColumn("quantity", col("quantity").cast("int"))
    .withColumn("unit_price", col("unit_price").cast("double"))
    .withColumn("total_price", col("quantity") * col("unit_price"))
    .withColumn("country", trim(upper(col("country"))))
    .filter(col("status") == "Delivered")
)
df_silver.write.format("delta").mode("overwrite").save("/mnt/datalake/silver/orders_clean")
```

## 🧩 5️⃣ Modelagem Dimensional (Camada Gold)

### 🎯 Fato e Dimensões

| Tabela          | Chave        | Descrição               |
| --------------- | ------------ | ----------------------- |
| **Fato_Vendas** | `sk_venda`   | Métricas das transações |
| **Dim_Cliente** | `sk_cliente` | Clientes únicos         |
| **Dim_Produto** | `sk_produto` | Produtos únicos         |
| **Dim_Tempo**   | `sk_data`    | Datas de venda          |
| **Dim_Pais**    | `sk_pais`    | Países padronizados     |

---

### 🧮 Dimensões

#### 🧍 Dim_Cliente

| Campo       | Tipo   |
| ----------- | ------ |
| sk_cliente  | INT    |
| customer_id | STRING |

#### 📦 Dim_Produto

| Campo      | Tipo   |
| ---------- | ------ |
| sk_produto | INT    |
| product_id | STRING |
| unit_price | DOUBLE |

#### 📅 Dim_Tempo

| Campo      | Tipo |
| ---------- | ---- |
| sk_data    | INT  |
| order_date | DATE |
| ano        | INT  |
| mes        | INT  |
| dia        | INT  |

#### 🌎 Dim_Pais

| Campo   | Tipo   |
| ------- | ------ |
| sk_pais | INT    |
| country | STRING |

---

### 💰 Fato_Vendas

| Campo       | Tipo   | Descrição   |
| ----------- | ------ | ----------- |
| sk_venda    | INT    | PK          |
| sk_cliente  | INT    | FK cliente  |
| sk_produto  | INT    | FK produto  |
| sk_data     | INT    | FK data     |
| sk_pais     | INT    | FK país     |
| quantity    | INT    | Quantidade  |
| total_price | DOUBLE | Valor total |

## 🪄 6️⃣ Código exemplo - Criação das dimensões e fato

```python
from pyspark.sql.functions import monotonically_increasing_id, year, month, dayofmonth

# Carregar camada Silver
df_silver = spark.read.format("delta").load("/mnt/datalake/silver/orders_clean")

# Dimensões
dim_cliente = df_silver.select("customer_id").distinct().withColumn("sk_cliente", monotonically_increasing_id())
dim_produto = df_silver.select("product_id", "unit_price").distinct().withColumn("sk_produto", monotonically_increasing_id())
dim_pais = df_silver.select("country").distinct().withColumn("sk_pais", monotonically_increasing_id())
dim_data = (
    df_silver.select("order_date")
    .distinct()
    .withColumn("ano", year("order_date"))
    .withColumn("mes", month("order_date"))
    .withColumn("dia", dayofmonth("order_date"))
    .withColumn("sk_data", monotonically_increasing_id())
)

# Fato
df_fact = (
    df_silver
    .join(dim_cliente, "customer_id", "left")
    .join(dim_produto, ["product_id", "unit_price"], "left")
    .join(dim_pais, "country", "left")
    .join(dim_data, "order_date", "left")
    .select(
        "sk_cliente", "sk_produto", "sk_pais", "sk_data",
        "quantity", "total_price"
    )
)

# Escrita Gold
dim_cliente.write.format("delta").mode("overwrite").save("/mnt/datalake/gold/dim_customer")
dim_produto.write.format("delta").mode("overwrite").save("/mnt/datalake/gold/dim_product")
dim_pais.write.format("delta").mode("overwrite").save("/mnt/datalake/gold/dim_country")
dim_data.write.format("delta").mode("overwrite").save("/mnt/datalake/gold/dim_date")
df_fact.write.format("delta").mode("overwrite").save("/mnt/datalake/gold/fact_sales")
```

## ✅ 7️⃣ Possíveis Requisitos Extras

Você pode adicionar para deixar o projeto mais completo:

| Tipo                   | Requisito                                                    |
| ---------------------- | ------------------------------------------------------------ |
| **Qualidade de dados** | Implementar checagem de nulos e logs de rejeição             |
| **Particionamento**    | `partitionBy("ano","mes")` em tabelas gold                   |
| **Agendamento**        | Executar via **Job do Databricks** diariamente               |
| **Monitoramento**      | Criar uma tabela de controle de ingestão (`metadata_ingest`) |
| **Alertas**            | Enviar erro no Slack/Teams quando a ingestão falhar          |
